```bash
gcloud compute instances create openmetadata \
--zone=europe-west9-b \
--machine-type=e2-medium \
--image-family=debian-11 \
--image-project=debian-cloud \
--boot-disk-size=50GB \
--scopes=https://www.googleapis.com/auth/cloud-platform \
--service-account=1034929772164-compute@developer.gserviceaccount.com
```

```bash
sudo apt update
sudo apt install apt-transport-https ca-certificates curl gnupg lsb-release
sudo mkdir -p /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/debian/gpg | sudo gpg --dearmor -o /etc/apt/keyrings/docker.gpg
echo "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/debian $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt update
sudo apt install docker-ce docker-ce-cli containerd.io docker-compose-plugin
docker --version
sudo usermod -aG docker $USER
```

```bash
mkdir openmetadata-docker && cd openmetadata-docker
```

```bash
curl -sL -o docker-compose.yml https://github.com/open-metadata/OpenMetadata/releases/download/1.6.8-release/docker-compose.yml

curl -sL -o docker-compose-postgres.yml https://github.com/open-metadata/OpenMetadata/releases/download/1.6.8-release/docker-compose-postgres.yml
```

```bash
wget https://github.com/open-metadata/OpenMetadata/releases/download/1.6.8-release/docker-compose.yml

wget https://github.com/open-metadata/OpenMetadata/releases/download/1.6.8-release/docker-compose-postgres.yml
```

```bash
docker compose -f docker-compose.yml up --detach
```

## Crée une règle de pare-feu pour ouvrir le port 8585 :

```bash
gcloud compute firewall-rules create allow-openmetadata \
  --allow=tcp:8585 \
  --source-ranges=0.0.0.0/0 \
  --target-tags=openmetadata-server \
  --description="Allow external access to OpenMetadata port 8585"
```

```bash
gcloud compute instances add-tags NOM_INSTANCE \
  --tags=openmetadata-server
```

http://<adresse_ip_publique>:8585

## Trouver le mot de passe de la DB

```bash
grep -r "MYSQL_ROOT_PASSWORD" .
```

## Trouver le nom du conteneur MySQL 

```bash
docker ps
```
> openmetadata_mysql

## Création du programme de push du fichier de persistance des données

```bash
touch backup_openmetadata.sh
vim backup_openmetadata.sh
```

```shell
#!/bin/bash

# Infos base de données
DB_NAME="openmetadata_db"
DB_USER="root"
DB_PASSWORD="password"
DATE=$(date +"%Y%m%d_%H%M")
BACKUP_NAME="openmetadata_${DATE}.sql"
LOCAL_PATH="$HOME/openmetadata_backups"
GCS_BUCKET="gs://werner-openmetadata"
DOCKER_CONTAINER_NAME="openmetadata_mysql"

# Créer dossier local si besoin
mkdir -p "$LOCAL_PATH"

# Dump dans le conteneur et envoie dans un fichier local
docker exec $DOCKER_CONTAINER_NAME sh -c "mysqldump -u $DB_USER -p$DB_PASSWORD $DB_NAME" > "$LOCAL_PATH/$BACKUP_NAME"

# Compression
gzip "$LOCAL_PATH/$BACKUP_NAME"

# Upload vers GCS
gsutil cp "$LOCAL_PATH/$BACKUP_NAME.gz" "$GCS_BUCKET/"

# Suppression locale
rm "$LOCAL_PATH/$BACKUP_NAME.gz"
```

## Déplacement et donner les droits d’exécution :
```bash
sudo mv backup_openmetadata.sh /usr/local/bin/backup_openmetadata.sh
sudo chmod +x /usr/local/bin/backup_openmetadata.sh
```

## Automatiser avec cron
```bash
crontab -e
```

```cron
0 2 * * * /usr/local/bin/backup_openmetadata.sh >> /var/log/openmetadata_backup.log 2>&1
```

## Tester que tout fonctionne correctement 

```bash
/usr/local/bin/backup_openmetadata.sh
```